In [1]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn.cluster
import tensorflow as tf

#from geoband import API

import pydeck as pdk
import os

import pandas as pd


import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

import numpy as np
from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY  


In [2]:
#Pydeck 사용을 위한 함수 정의
import geopandas as gpd 
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

In [20]:
# 격자별 인구 현황
df_08= gpd.read_file("C:/Users/user/Desktop/분석데이터.geojson") #데이터

df_08.columns

Index(['gid', '50대남', '50대여', '60대남', '60대여', '70대남', '70대여', '80대남', '80대여', '90대남', '90대여', '100대남', '100대여', '정보화�', '도봉구_배움터,총교육,이용률_배움터', '도봉구_배움터,총교육,이용률_이용률', '도봉구_배움터,총교육,이용률_총교육', 'geometry'], dtype='object')

In [29]:
# 격자별 인구 현황
df_08= gpd.read_file("C:/Users/user/Desktop/분석데이터.geojson") #데이터

df_08 = df_08.fillna(0)
# val 열 na 제거

df_08['총인구수'] = df_08['50대남']+df_08['50대여']+df_08['60대남']+df_08['60대여']+df_08['70대남']+df_08['70대여']+df_08['80대남']+df_08['80대여']+df_08['90대남']+df_08['90대여']+df_08['100대남']+df_08['100대여']

df_08['총인구수'] = df_08['총인구수'].fillna(0)
        
# 인구 수 정규화
df_08['정규화인구'] = df_08['총인구수'] / df_08['총인구수'].max()

df_08.rename(columns={"정보화�":"정보화점수",
                     '도봉구_배움터,총교육,이용률_배움터':'디지털배움터수',
                     '도봉구_배움터,총교육,이용률_총교육':'총교육수',
                     '도봉구_배움터,총교육,이용률_이용률':'이용률'},inplace=True)

df_08 = df_08[['gid','총인구수','디지털배움터수','총교육수','이용률',
      '정보화점수','정규화인구','geometry']]


# geometry를 coordinate 형태로 적용
df_08['coordinates'] = df_08['geometry'].apply(multipolygon_to_coordinates) #pydeck 을 위한 coordinate type

# 100X100 grid에서 central point 찾기
df_08_list = []
df_08_list2 = []
for i in df_08['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    df_08_list.append(cent)
    df_08_list2.append(Point(cent[0]))
df_08['coord_cent'] = 0
df_08['geo_cent'] = 0
df_08['coord_cent']= pd.DataFrame(df_08_list) # pydeck을 위한 coordinate type
df_08['geo_cent'] = df_08_list2 # geopandas를 위한 geometry type


# 쉬운 분석을 위한 임의의 grid id 부여
df_08['grid_id']=0
idx = []
for i in range(len(df_08)):
    idx.append(str(i).zfill(5))
df_08['grid_id'] = pd.DataFrame(idx)


# 인구 현황이 가장 높은 위치
df_08.iloc[df_08["정규화인구"].sort_values(ascending=False).index].reindex().head()


,gid,총인구수,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,geometry,coordinates,coord_cent,geo_cent,grid_id
567,다사586620,768.0,0.0,0,0,75.286,1.000000,"MULTIPOLYGON (((127.03064 37.65657, 127.03064 ...","[[127.03064121039422, 37.65656604074309], [127...","[127.03120521792778, 37.65701895053724]",POINT (127.0312052179278 37.65701895053724),00567
634,다사596616,631.0,0.0,0,0,73.017,0.821615,"MULTIPOLYGON (((127.04200 37.65301, 127.04199 ...","[[127.04200018787387, 37.65300534794231], [127...","[127.04256423826097, 37.65345820397527]",POINT (127.042564238261 37.65345820397527),00634
184,다사602648,614.0,0.0,0,0,69.450,0.799479,"MULTIPOLYGON (((127.04863 37.68187, 127.04862 ...","[[127.04862733199549, 37.68187357593071], [127...","[127.04919163970389, 37.682326399727785]",POINT (127.0491916397039 37.68232639972778),00184
219,다사581623,603.0,0.0,0,0,73.101,0.785156,"MULTIPOLYGON (((127.02496 37.65925, 127.02495 ...","[[127.0249555755653, 37.659247284714006], [127...","[127.02551956837783, 37.65970022139157]",POINT (127.0255195683778 37.65970022139157),00219
305,다사602651,598.0,0.0,0,0,74.227,0.778646,"MULTIPOLYGON (((127.04861 37.68458, 127.04861 ...","[[127.04861094687551, 37.68457750766119], [127...","[127.04917527480133, 37.68503033146032]",POINT (127.0491752748013 37.68503033146032),00305


In [25]:
MAPBOX_API_KEY = 'pk.eyJ1IjoiYmJoaCIsImEiOiJja3M3aWswOGEwNzUwMnVrMXkyZGV4Ym41In0.vSNCsuT3Ov428QmTyGXILQ'

In [30]:
# Make layer
# 사람이 있는 그리드만 추출
layer = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_08[(df_08['총인구수'].isnull()==False) & df_08['총인구수']!=0], # 시각화에 쓰일 데이터프레임 
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[0, 255*정규화인구, 0, 정규화인구*10000 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 )

# Set the viewport location 
center = [127.0323669951, 37.6691052785] 
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 

# Render 
r = pdk.Deck(layers=[layer], 
             map_style='mapbox://styles/mapbox/outdoors-v11',
             mapbox_key = MAPBOX_API_KEY,
             initial_view_state=view_state
            )
r.to_html()

In [33]:
df_08.columns
df_result = df_08

#'도봉구_배움터,총교육,이용률_배움터','도봉구_배움터,총교육,이용률_이용률','도봉구_배움터,총교육,이용률_총교육'

df_result['정규화_디지털배움터수'] = df_result['디지털배움터수'] / df_result['디지털배움터수'].max()
df_result['정규화_이용률'] = df_result['이용률'] / df_result['이용률'].max()
df_result['정규화_총교육수'] = df_result['총교육수'] / df_result['총교육수'].max()
df_result['정규화_정보화점수'] = df_result['정보화점수'] / df_result['정보화점수'].max()

df_result.head()

,gid,총인구수,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,geometry,coordinates,coord_cent,geo_cent,grid_id,정규화_디지털배움터수,정규화_이용률,정규화_총교육수,정규화_정보화점수
0,다사594644,0.0,0.0,0,0,0.000,0.000000,"MULTIPOLYGON (((127.03958 37.67823, 127.03957 ...","[[127.03957706314256, 37.678233263718006], [12...","[127.04014128804377, 37.67868613055946]",POINT (127.0401412880438 37.67868613055946),00000,0.0,0.0,0.0,0.000000
1,다사593608,202.0,0.0,0,0,75.861,0.263021,"MULTIPOLYGON (((127.03864 37.64578, 127.03864 ...","[[127.03864390692696, 37.64578157893305], [127...","[127.03920788259077, 37.64623445107446]",POINT (127.0392078825908 37.64623445107446),00001,0.0,0.0,0.0,0.736515
2,다사591623,212.0,0.0,0,0,71.627,0.276042,"MULTIPOLYGON (((127.03629 37.65929, 127.03629 ...","[[127.03629280657076, 37.65929239742478], [127...","[127.03685686917244, 37.65974528035168]",POINT (127.0368568691724 37.65974528035168),00002,0.0,0.0,0.0,0.695408
3,다사596628,26.0,0.0,0,0,46.000,0.033854,"MULTIPOLYGON (((127.04193 37.66382, 127.04193 ...","[[127.04193374093843, 37.66382111048692], [127...","[127.04249787207377, 37.664273966540264]",POINT (127.0424978720738 37.66427396654026),00003,0.0,0.0,0.0,0.446602
4,다사585609,40.0,0.0,0,0,79.850,0.052083,"MULTIPOLYGON (((127.02957 37.64665, 127.02956 ...","[[127.02957007554765, 37.646647081461836], [12...","[127.03013400214626, 37.647099996590754]",POINT (127.0301340021463 37.64709999659075),00004,0.0,0.0,0.0,0.775243


In [35]:
df_result['w_FS'] = 0 
df_result['w_FS'] = (0.2686*df_result['정규화인구']+
                     0.3115*df_result['정규화_디지털배움터수']+
                     -0.2612*df_result['정규화_이용률']+
                     -0.1526*df_result['정규화_총교육수']+
                     21.3516*df_result['정규화_정보화점수']
                    )
df_result.columns

Index(['gid', '총인구수', '디지털배움터수', '총교육수', '이용률', '정보화점수', '정규화인구', 'geometry', 'coordinates', 'coord_cent', 'geo_cent', 'grid_id', '정규화_디지털배움터수', '정규화_이용률', '정규화_총교육수', '정규화_정보화점수', 'w_FS'], dtype='object')

In [9]:
df_result[['gid','디지털배움터수','총교육수','이용률','정보화점수','정규화인구','geometry','grid_id','정규화_디지털배움터수','정규화_이용률','정규화_총교육수','정규화_정보화점수','w_FS']].to_file("C:/Users/user/Desktop/df_result.geojson",driver="GeoJSON",encoding='utf-8')

In [36]:
def generate_candidate_sites(points,M=100):
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        points: a Numpy array with shape of (N,2)
        M: the number of candidate sites to generate
    Return:
        sites: a Numpy array with shape of (M,2)
    '''
    hull = ConvexHull(points)
    polygon_points = points[hull.vertices]
    poly = Polygon(polygon_points)
    min_x, min_y, max_x, max_y = poly.bounds
    sites = []
    while len(sites) < M:
        random_point = Point([random.uniform(min_x, max_x),
                             random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])




def generate_candidate_sites(df_result_fin,M=100):
    from shapely.geometry import Polygon, Point
    sites = []
    idx=np.random.choice(np.array(range(0,len(df_result_fin))), M)
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.iloc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.iloc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])

def generate_candidate_sites(df_result_fin,Weight,M=100):
    sites = []
    idx = df_result_fin.sort_values(by = Weight, ascending = False).iloc[1:M].index
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.loc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.loc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])



from scipy.spatial import distance_matrix
def mclp(points,K,radius,M,df_result_fin,w,Weight):

    """
    Solve maximum covering location problem
    Input:
        points: input points, Numpy array in shape of [N,2]
        K: the number of sites to select
        radius: the radius of circle
        M: the number of candidate sites, which will randomly generated inside
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    print('----- Configurations -----')
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)
    print('  M %g' % M)
    import time
    start = time.time()
    sites = generate_candidate_sites(df_result_fin,Weight,M)
    J = sites.shape[0]
    I = points.shape[0]
    D = distance_matrix(points,sites)
    mask1 = D<=radius
    D[mask1]=1
    D[~mask1]=0

    from mip import Model, xsum, maximize, BINARY

    # Build model
    m = Model("mclp")
    # Add variables

    x = [m.add_var(name = "x%d" % j, var_type = BINARY) for j in range(J)]
    y = [m.add_var(name = "y%d" % i, var_type = BINARY) for i in range(I)]


    m.objective = maximize(xsum(w[i]*y[i] for i in range (I)))

    m += xsum(x[j] for j in range(J)) == K

    for i in range(I):
        m += xsum(x[j] for j in np.where(D[i]==1)[0]) >= y[i]

    m.optimize()
    
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objective_value)

    solution = []
    for i in range(J):
        if x[i].x ==1:
            solution.append(int(x[i].name[1:]))
    opt_sites = sites[solution]
            
    return opt_sites,m.objective_value




In [38]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#import xgboost
import sklearn.cluster
import tensorflow as tf

#from geoband import API

import pydeck as pdk
import os

import pandas as pd


import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
#import deckgljupyter.Layer as deckgl

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

import numpy as np
from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY  

df_test=gpd.read_file('C:/Users/user/Desktop/df_result.geojson')
df_test
# 100X100 grid에서 central point 찾기
df_list = []
df_list2 = []
for i in df_test['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    df_list.append(cent)
    df_list2.append(Point(cent[0]))
df_test['coord_cent'] = 0
df_test['geo_cent'] = 0
df_test['coord_cent']= pd.DataFrame(df_list) # pydeck을 위한 coordinate type
df_test['geo_cent'] = df_list2 # geopandas를 위한 geometry type

0       0.000000
1       0.341927
2       0.358854
3       0.044010
4       0.067708
          ...   
2221    0.000000
2222    0.000000
2223    0.000000
2224    0.000000
2225    0.000000
Name: w_FS, Length: 2226, dtype: float64

In [39]:
df_result_fin = df_test
df_result_fin

points = []
for i in df_result_fin['coord_cent'] :
    points.append(i)

w= []
for i in df_result_fin['w_FS'] :
    w.append(i)

radius = radius = (1/88.74/1000)*300  #반경?   
K = 10     # 입지 선정 개수
M = 5000  #후보 입지 수

opt_sites_org,f = mclp(np.array(points),K,radius,M,df_result_fin,w,'w_FS')


df_opt_FS= pd.DataFrame(opt_sites_org)
df_opt_FS.columns = ['lon', 'lat']
df_opt_FS

----- Configurations -----
  Number of points 2226
  K 10
  Radius 0.00338066
  M 5000
----- Output -----
  Running time : 6.59972882270813 seconds
  Optimal coverage points: 117.35


,lon,lat
0,127.025525,37.658799
1,127.042553,37.655261
2,127.030128,37.648001
3,127.039080,37.666965
4,127.031200,37.657920
5,127.036862,37.658844
6,127.047071,37.657982
7,127.050527,37.648982
8,127.035830,37.642616
9,127.046913,37.684120


In [40]:
df_opt_FS.to_csv('C:/Users/user/Desktop/결과.csv',index=False)